# 一、贝叶斯分类器简单介绍

朴素贝叶斯分类器是一种基于贝叶斯定理的简单概率分类器，广泛应用于机器学习领域中的分类任务。它之所以被称为“朴素”，是因为它假设所有特征之间相互独立，即给定目标类别，一个特征出现的概率不影响任何其他特征的出现概率。尽管这一假设在现实世界中往往不成立，但朴素贝叶斯分类器在许多情况下仍然能够提供强大的分类性能。

### 贝叶斯定理

朴素贝叶斯分类器的核心是贝叶斯定理，它描述了在给定相关证据或信息的情况下，某个假设的概率是如何变化的。贝叶斯定理的数学表达式如下：

$$ P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)} $$

其中：
- $ P(A|B) $ 是在事件$B$发生的条件下事件$A$发生的概率，称为后验概率。
- $ P(B|A) $ 是在事件$A$发生的条件下事件$B$发生的概率，称为似然概率。
- $ P(A) $ 是事件$A$的先验概率。
- $ P(B) $ 是事件$B$的边缘概率。

### 朴素贝叶斯分类器的工作原理

朴素贝叶斯分类器的目标是找到给定一组特征$X$（例如，特征向量或数据实例）的条件下，每个类别$Y$的概率，即计算$ P(Y|X) $。根据贝叶斯定理，我们可以将这个概率表示为：

$$ P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)} $$

由于我们通常对类别$Y$的分布更感兴趣，而不是特征$X$的分布，我们可以忽略分母$ P(X) $，因为它对于所有类别是相同的。因此，我们可以简化为：

$$ P(Y|X) \propto P(X|Y) \cdot P(Y) $$

这意味着我们可以通过比较不同类别下$ P(X|Y) $和$ P(Y) $的乘积来确定最可能的类别。

### 特征独立性假设

朴素贝叶斯分类器的关键假设是特征之间的独立性。假设我们有n个特征$( X_1, X_2, ..., X_n )$，那么给定类别$Y$的条件下，特征联合概率可以表示为特征概率的乘积：

$$ P(X_1, X_2, ..., X_n|Y) = P(X_1|Y) \cdot P(X_2|Y) \cdot ... \cdot P(X_n|Y) $$

这个假设简化了模型的计算，使其在实际应用中更加高效。尽管这一假设可能不完全准确，但朴素贝叶斯分类器在许多情况下仍然能够提供良好的性能，这可能是因为它能够捕捉到特征和类别之间的条件依赖关系。

### 训练和预测

在训练阶段，朴素贝叶斯分类器需要计算以下两个概率：
1. **先验概率** $P(Y) $：每个类别在训练数据中出现的频率。
2. **条件概率** $ P(X_i|Y) $：给定类别$Y$的条件下，每个特征$X_i$出现的概率。对于连续特征，这通常通过计算特征在每个类别中的平均值和方差来估计高斯分布；对于离散特征，这可以通过计算特征在每个类别中出现的频率来估计。

在预测阶段，对于一个新的实例$X$，分类器计算每个类别下特征的概率乘积，然后选择具有最高后验概率的类别作为预测结果。

### 应用场景

朴素贝叶斯分类器适用于多种应用场景，包括但不限于：
- 文本分类（如垃圾邮件检测、情感分析）
- 医疗诊断
- 客户分类
- 推荐系统

### 优点和局限性

**优点**：
- 简单、易于实现
- 计算效率高，适合大规模数据集
- 需要的样本量相对较少
- 可以处理高维数据

**局限性**：
- 特征独立性假设可能不成立，可能影响分类性能
- 对于连续特征，需要假设概率分布（如高斯分布），这可能不适用于所有数据
- 对于不平衡数据集，可能需要进行额外的平衡处理

尽管存在局限性，朴素贝叶斯分类器仍然是一种强大且广泛使用的工具，特别是在文本分类和自然语言处理领域。

# 二、代码详解

In [1]:
import numpy as np
import pandas as pd

In [22]:
# 原ppt内容
data = [['A','S',0],['A','M',0],['A','M',1],['A','S',1],\
        ['A','S',0],['B','S',0],['B','M',0],['B','M',1],\
        ['B','L',1],['B','L',1],['C','L',1],['C','M',1], \
        ['C','M',1],['C','L',1],['C','L',0]]

对表格内容进行读取，以下为表格内容截图

In [48]:
# 读取表格内容
data = pd.read_excel(r'data.xlsx', sheet_name=0)

![](2024-04-15-21-07-20.png)

In [58]:
Data = pd.DataFrame(data,columns= ['Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis'])
Data.head(14)

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [59]:
cols=Data.shape[1] #5列：4个特征+1个标签
X_data = Data.iloc[:,:cols-1]
Y_data = Data.iloc[:,cols-1:]
featureNames = X_data.columns
X_data,Y_data

(     Outlook Temperature Humidity    Wind
 0      Sunny         Hot     High    Weak
 1      Sunny         Hot     High  Strong
 2   Overcast         Hot     High    Weak
 3       Rain        Mild     High    Weak
 4       Rain        Cool   Normal    Weak
 5       Rain        Cool   Normal  Strong
 6   Overcast        Cool   Normal  Strong
 7      Sunny        Mild     High    Weak
 8      Sunny        Cool   Normal    Weak
 9       Rain        Mild   Normal    Weak
 10     Sunny        Mild   Normal  Strong
 11  Overcast        Mild     High  Strong
 12  Overcast         Hot   Normal    Weak
 13      Rain        Mild     High  Strong,
    PlayTennis
 0          No
 1          No
 2         Yes
 3         Yes
 4         Yes
 5          No
 6         Yes
 7          No
 8         Yes
 9         Yes
 10        Yes
 11        Yes
 12        Yes
 13         No)

In [79]:
# 定义 Naive Bayes 函数  
def Naive_Bayes(X_data, Y_data):  
    # 步骤 1: 计算先验概率  
    # 将 Y_data 转换为 numpy 数组，方便后续计算  
    y = Y_data.values  
    # 将 X_data 转换为 numpy 数组  
    X = X_data.values  
    # 获取 Y_data 中所有不同的标签值，即No和Yes
    y_unique = np.unique(y)  
    # 初始化先验概率数组，长度为不同的标签值的数量  
    prior_prob = np.zeros(len(y_unique))  
    # 遍历每一个不同的标签值  
    for i in range(len(y_unique)):  
        # 计算当前标签值的先验概率（出现频率），即出现No的概率和Yes的概率是多少
        prior_prob[i] = sum(y == y_unique[i]) / len(y) 
    
    # 步骤 2: 计算似然概率（条件概率）  
    # 初始化条件概率的字典  
    condition_prob = {}  
    # 遍历特征名列表中的每一个特征  
    for feat in featureNames:  # featureNames指的就是'Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis'
        # 获取当前特征的所有不同取值  
        x_unique = list(set(X_data[feat]))  
        # 初始化条件概率矩阵，行数为不同的标签值的数量，列数为当前特征的不同取值数量  
        x_condition_prob = np.zeros((len(y_unique), len(x_unique)))
        # 遍历每一个不同的标签值  
        for j in range(len(y_unique)):  
            # 遍历当前特征的所有不同取值  
            for k in range(len(x_unique)):  
                # 计算当前标签值和当前特征取值下的条件概率  
                x_condition_prob[j, k]= \
                sum((X_data[feat] == x_unique[k]) & (Y_data.PlayTennis == y_unique[j])) / sum(y == y_unique[j])  
        # 将计算得到的条件概率矩阵转换为 DataFrame 格式，方便后续处理  
        x_condition_prob = pd.DataFrame(x_condition_prob, columns=x_unique, index=y_unique)  
        # 将当前特征的条件概率矩阵添加到条件概率字典中  
        condition_prob[feat] = x_condition_prob  
    # 返回先验概率和条件概率  
    return prior_prob, condition_prob

In [80]:
prior_prob,condition_prob=Naive_Bayes(X_data,Y_data)
print (prior_prob)
print (condition_prob['Outlook'])
print (condition_prob['Temperature'])
print (condition_prob['Humidity'])
print (condition_prob['Wind'])

[0.35714286 0.64285714]
        Sunny  Overcast      Rain
No   0.600000  0.000000  0.400000
Yes  0.222222  0.444444  0.333333
          Hot      Mild      Cool
No   0.400000  0.400000  0.200000
Yes  0.222222  0.444444  0.333333
       Normal      High
No   0.200000  0.800000
Yes  0.666667  0.333333
         Weak    Strong
No   0.400000  0.600000
Yes  0.666667  0.333333


In [68]:
def Prediction(testData, prior, condition_prob):  
    # 获取类别的数量，由先验概率数组的行数决定  
    numclass = prior.shape[0]  
    # 获取测试数据的特征名  
    featureNames = testData.columns  
    # 获取测试数据的样本数量  
    numsample = testData.shape[0]  
      
    # 初始化一个numsample行numclass列的零矩阵，用于存储后验概率  
    post_prob = np.zeros((numsample, numclass))  
      
    # 遍历每一个测试样本  
    for k in range(numsample):  
        # 初始化一个长度为numclass的零数组，用于存储当前样本在每个类别下的概率乘积  
        prob_k = np.zeros((numclass,))  
          
        # 遍历每一个类别  
        for i in range(numclass):  
            # 如果当前类别是第一个类别，结果设置为'No'  
            if i == 0:  
                result = 'No'  
            # 如果当前类别是第二个类别，结果设置为'Yes'  
            elif i == 1:  
                result = 'Yes'  
            # 获取当前类别的先验概率  
            pri = prior[i]  
              
            # 遍历每一个特征  
            for feat in featureNames:  
                # 获取当前样本在当前特征下的取值  
                feat_val = testData[feat][k]  
                # 获取当前类别在当前特征下的条件概率表  
                cp = condition_prob[feat]  
                # 获取当前样本在当前特征取值下的条件概率  
                cp_val = cp.loc[result, feat_val]  
                # 将当前类别的先验概率与条件概率相乘，更新当前类别的概率乘积  
                pri *= cp_val  
              
            # 将当前类别的概率乘积存入prob_k数组  
            prob_k[i] = pri  
          
        # 将prob_k数组归一化，得到当前样本在每个类别下的后验概率  
        prob = prob_k / np.sum(prob_k, axis=0)  
        # 将当前样本的后验概率存入post_prob矩阵  
        post_prob[k, :] = prob  
      
    # 返回所有样本的后验概率矩阵  
    return post_prob

In [81]:
test_data=[['Sunny','Cool','High','Strong']]
testData=pd.DataFrame(test_data,columns=[ 'Outlook', 'Temperature', 'Humidity', 'Wind'])
testData.head()
postPrior = Prediction(testData,prior_prob,condition_prob)
postPrior

array([[0.79541735, 0.20458265]])

In [82]:
if postPrior[0][0]>postPrior[0][1]:
    print('No')
else:
    print('Yes')

No
